In [ ]:
# import easy slurm
import datetime
import os
import shutil
import time
from pathlib import Path
from subprocess import check_output

from simple_slurm import Slurm

In [ ]:
input_path = "/cluster/project/treutlein/DATA/imaging/4i_Data/Brain_ECM_4i_2_AGAR/"
segment_channel = 2
segment_cycle = 1
pannel = "4i_ecm_brain_organoids_2.csv"
cutter_size = 400

In [ ]:
dirlist = [
    filename
    for filename in os.listdir(input_path)
    if (os.path.isdir(os.path.join(input_path, filename)) and filename != "raw")
]
dirlist = [filename for filename in dirlist if filename != "cellpose_model"]
# Cleanup input dir
for dir_name in dirlist:
    dir_path = os.path.join(input_path, dir_name)
    try:
        print(f"Deleting {dir_path}")
        shutil.rmtree(dir_path)

    except:
        print(f"Already deleted: {dir_path}")

In [ ]:
# Run prepare_pipeline
slurm = Slurm(
    cpus_per_task=4,
    mem_per_cpu=5000,
    job_name=f"prepare_4i",
    output=f"logs/job_%j.out",
    account="es_treutlein",
    time=datetime.timedelta(days=0, hours=4, minutes=0, seconds=0),
)

slurm.add_cmd("source activate phenoscapes_main")
job_name_prepare = slurm.sbatch(
    f"python pipeline/prepare_pipeline_compartment_analysis.py --input_path {input_path}"
    + f" --segment_channel {segment_channel}"
    + f" --segment_cycle {segment_cycle}"
    + f" --cutter_size {cutter_size}"
    + f" --pannel {pannel}",
    shell="/bin/bash",
)


# Extract job ID
job_id_prepare = job_name_prepare

# Wait for the first job to complete
job_status = "PENDING"
while "COMPLETED" not in job_status:
    try:

        job_status = (
            check_output(
                f"sacct -j {job_id_prepare} --format=State --noheader", shell=True
            )
            .decode()
            .strip()
        )
        print(job_status)
    except Exception as e:
        print(f"Error checking job status: {e}")
    time.sleep(60)  # Wait for 1 minute before checking again

if "COMPLETED" in job_status:
    # For len dirs in stitched dir run pipeline as a job array
    dir_stitched = Path(input_path, "stitched")
    well_id = os.listdir(dir_stitched)
    # Run single cell segmentation --> Job array
    slurm = Slurm(
        array=range(len(well_id)),
        cpus_per_task=2,
        mem_per_cpu=12000,
        dependency=dict(afterok=job_name_prepare),
        job_name="Phenoscapes",
        output=f"logs/job_%j.out",
        error=f"error/job_%j.out",
        account="es_treutlein",
        time=datetime.timedelta(days=0, hours=12, minutes=0, seconds=0),
    )

    slurm.add_cmd("source activate phenoscapes_main")
    job_name_phenoscapes = slurm.sbatch(
        f"python pipeline/run_phenoscapes.py --input_path {input_path} --array_num {Slurm.SLURM_ARRAY_TASK_ID}",
        shell="/bin/bash",
    )